In [9]:
raw_text = "A quick from brown fox jumps over a lazy dog"
norm_text = raw_text.lower().replace(" ", "$")
print(norm_text)
print("Total pairs with space, except double space", 27*27 - 1)

a$quick$from$brown$fox$jumps$over$a$lazy$dog
Total pairs with space, except double space 728


In [19]:
def normalize_text(text):
    return text.lower().lstrip().rstrip().replace("\n", " ").replace(" ", "$")

In [20]:
in_the_end_lyrics = """
It starts with one
[Mike Shinoda:]
One thing I don't know why
It doesn't even matter how hard you try
Keep that in mind, I designed this rhyme
To explain in due time
[Chester Bennington:]
All I know
[Mike Shinoda:]
Time is a valuable thing
Watch it fly by as the pendulum swings
Watch it count down to the end of the day
The clock ticks life away
[Chester Bennington:]
It's so unreal
[Mike Shinoda:]
It's so unreal, didn't look out below
Watch the time go right out the window
Trying to hold on, but didn't even know
I wasted it all just to watch you go
[Chester Bennington:]
Watch you go
[Mike Shinoda:]
I kept everything inside and even though I tried, it all fell apart
What it meant to me will eventually be a memory of a time when I tried so hard
[Chester Bennington:]
I tried so hard
And got so far
But in the end
It doesn't even matter
I had to fall
To lose it all
But in the end
It doesn't even matter
[Mike Shinoda:]
One thing, I don't know why
It doesn't even matter how hard you try
Keep that in mind, I designed this rhyme
To remind myself how I tried so hard
[Chester Bennington:]
I tried so hard
[Mike Shinoda:]
In spite of the way you were mocking me
Acting like I was part of your property
Remembering all the times you fought with me
I'm surprised it got so far
[Chester Bennington:]
Got so far
[Mike Shinoda:]
Things aren't the way they were before
You wouldn't even recognize me anymore
Not that you knew me back then
But it all comes back to me in the end
[Chester Bennington:]
In the end
[Mike Shinoda:]
You kept everything inside and even though I tried, it all fell apart
What it meant to me will eventually be a memory of a time when I tried so hard
[Chester Bennington:]
I tried so hard
And got so far
But in the end
It doesn't even matter
I had to fall
To lose it all
But in the end
It doesn't even matter
I've put my trust in you
Pushed as far as I can go
For all this
There's only one thing you should know
I've put my trust in you
Pushed as far as I can go
For all this
There's only one thing you should know
I tried so hard
And got so far
But in the end
It doesn't even matter
I had to fall
To lose it all
But in the end
It doesn't even matter
"""

norm_text = normalize_text(in_the_end_lyrics.replace("[Chester Bennington:]", "").replace("[Mike Shinoda:]",""))
print(len(norm_text))

1885


In [35]:
def get_bigrams(text, delta = None):
    if delta is None:
        delta = [1, 2, 3, 4, 5, 6, 7, 8]
    global_bigrams = set()
    for d in delta:
        bigrams = []
        for i in range(len(text)-d):
            bigrams.append((text[i], text[i+d]))
            global_bigrams.add((text[i], text[i+d]))
            
        print(f'delta={d}\nbigrams={bigrams}')
        print(f"Length of global_bigrams: {len(global_bigrams)}\n")
        
    print(f'Length of pairs: {len(global_bigrams)}\nglobal_bigrams={global_bigrams}')
    return global_bigrams
        

In [36]:
src_down_time, src_up_time, dest_down_time, dest_up_time, delta, local_hour = (0, 0, 0, 0, 0, 0) #input
src_key, dest_key = ("", "") #output

In [37]:
import pandas as pd

# create dataframe from keystrokes.csv with heading as event_type, key, time
df = pd.read_csv("keystrokes.csv", header=None, names=["event_type", "key", "time"])
df.head()

,event_type,key,time
0,event_type,key,timestamp
1,D,$,1726245845614094400
2,U,$,1726245845837010600
3,D,$,1726245852925579000
4,D,I,1726245853057461600


In [39]:
seq_df = df[(df["event_type"] == "D")]
len(seq_df)

390

In [40]:
raw_input = ''.join(seq_df["key"].tolist())
bigrams = get_bigrams(raw_input)

delta=1
bigrams=[('$', '$'), ('$', 'I'), ('I', '@'), ('@', 't'), ('t', 'r'), ('r', 'i'), ('i', 'e'), ('e', 'd'), ('d', '@'), ('@', 's'), ('s', 'o'), ('o', '@'), ('@', 'h'), ('h', 'a'), ('a', 'r'), ('r', 'd'), ('d', '@'), ('@', 'a'), ('a', 'n'), ('n', 'd'), ('d', '@'), ('@', 'g'), ('g', 'o'), ('o', 't'), ('t', '@'), ('@', 's'), ('s', 'o'), ('o', '@'), ('@', 'f'), ('f', 'a'), ('a', 'r'), ('r', '@'), ('@', '$'), ('$', 'B'), ('B', 'u'), ('u', 't'), ('t', '@'), ('@', 'i'), ('i', 'n'), ('n', '@'), ('@', 't'), ('t', 'h'), ('h', 'e'), ('e', '@'), ('@', 'e'), ('e', 'n'), ('n', 'd'), ('d', '@'), ('@', 'i'), ('i', 't'), ('t', '@'), ('@', 'd'), ('d', 'o'), ('o', 'e'), ('e', 's'), ('s', '#'), ('#', 't'), ('t', '@'), ('@', 'e'), ('e', 'v'), ('v', 'e'), ('e', 'n'), ('n', '@'), ('@', 'm'), ('m', 'a'), ('a', 'y'), ('y', 'y'), ('y', 'e'), ('e', 'r'), ('r', '@'), ('@', '$'), ('$', 'O'), ('O', '@'), ('@', '$'), ('$', 'I'), ('I', '@'), ('@', 'h'), ('h', 'a'), ('a', 'd'), ('d', '@'), ('@', 't'), ('t', 'o'),

In [64]:
def get_temporal_keystrokes(df: pd.DataFrame):
    
    temporal_keystrokes = []
    last_key_idx = dict()
    first_down_encountered = False
    
    def norm_key(key):
        return key.lower() if key.isalpha() else key
    
    for row in df.itertuples(index = True):
        if not first_down_encountered and row.event_type == "U":
            continue
        nkey = norm_key(row.key)
        if row.event_type == "D":
            first_down_encountered = True
            last_key_idx[nkey] = len(temporal_keystrokes)
            temporal_keystrokes.append({"key": nkey, "down_time": int(row.time), "up_time": 0})
        else:
            temporal_keystrokes[last_key_idx[nkey]]["up_time"] = int(row.time) 
            
    return temporal_keystrokes

temporal_keystrokes = get_temporal_keystrokes(df[1:])
temporal_keystrokes

[{'key': '$',
  'down_time': 1726245845614094400,
  'up_time': 1726245845837010600},
 {'key': '$',
  'down_time': 1726245852925579000,
  'up_time': 1726245853262504900},
 {'key': 'i',
  'down_time': 1726245853057461600,
  'up_time': 1726245853208462700},
 {'key': '@',
  'down_time': 1726245853399054600,
  'up_time': 1726245853476904300},
 {'key': 't',
  'down_time': 1726245853692466100,
  'up_time': 1726245853840088200},
 {'key': 'r',
  'down_time': 1726245853912081300,
  'up_time': 1726245853993473100},
 {'key': 'i',
  'down_time': 1726245854060136000,
  'up_time': 1726245854168925300},
 {'key': 'e',
  'down_time': 1726245854142706300,
  'up_time': 1726245854252538500},
 {'key': 'd',
  'down_time': 1726245854360682700,
  'up_time': 1726245854444547200},
 {'key': '@',
  'down_time': 1726245854444547200,
  'up_time': 1726245854497480300},
 {'key': 's',
  'down_time': 1726245854609426200,
  'up_time': 1726245854693696400},
 {'key': 'o',
  'down_time': 1726245854715797200,
  'up_time': 17

In [82]:
def nano_to_hour(nano):
    return (nano // (1000_000_000 * 60 * 60) ) % 24

def create_training_data(temporal_keystrokes, delta = None):
    if delta is None:
        delta = [1, 2, 3, 4, 5, 6, 7, 8]
    X, Y = [], []
    for d in delta:
        for i in range(len(temporal_keystrokes)-d):
            X.append({
                'src_hold_time': temporal_keystrokes[i]["up_time"] - temporal_keystrokes[i]["down_time"],
                'dest_hold_time': temporal_keystrokes[i+d]["up_time"] - temporal_keystrokes[i+d]["down_time"],
                'flight_time': temporal_keystrokes[i+d]["down_time"] - temporal_keystrokes[i]["down_time"],
                'delta': d,
                'local_hour': nano_to_hour(temporal_keystrokes[i]["down_time"])
            })
            
            Y.append({
                'src_key': temporal_keystrokes[i]["key"],
                'dest_key': temporal_keystrokes[i+d]["key"]
            })
            
        
    print(f'Length of training set: {len(Y)}')
    return X, Y

In [83]:
X, Y = create_training_data(temporal_keystrokes)

Length of training set: 3084


In [84]:
X[1000:1005]

[{'src_hold_time': 82555000,
  'dest_hold_time': 82903500,
  'flight_time': 384971000,
  'delta': 3,
  'local_hour': 16},
 {'src_hold_time': 83678000,
  'dest_hold_time': 53742300,
  'flight_time': 331754200,
  'delta': 3,
  'local_hour': 16},
 {'src_hold_time': 83648500,
  'dest_hold_time': 168077500,
  'flight_time': 600073600,
  'delta': 3,
  'local_hour': 16},
 {'src_hold_time': 82903500,
  'dest_hold_time': 118331200,
  'flight_time': 568959500,
  'delta': 3,
  'local_hour': 16},
 {'src_hold_time': 53742300,
  'dest_hold_time': 54264600,
  'flight_time': 687414500,
  'delta': 3,
  'local_hour': 16}]

In [85]:
Y[1000:1005]

[{'src_key': 'm', 'dest_key': 'd'},
 {'src_key': 'i', 'dest_key': '@'},
 {'src_key': 'n', 'dest_key': '$'},
 {'src_key': 'd', 'dest_key': 'i'},
 {'src_key': '@', 'dest_key': '@'}]

In [89]:
def normalize_training_data(X, Y, max_hold_time_ms = 800, max_flight_time_ms = 1200):
    invert_map = { val: idx for (idx, val) in enumerate([ chr( ord('a') + i) for i in range(26)] + ["#", "$", "@"]) }
    Xnew, Ynew = [], []
    for idx in range(len(Y)):
        Xnew.append({
            'src_hold_time': X[idx]['src_hold_time'] / (1000_000 * max_hold_time_ms),
            'dest_hold_time': X[idx]['dest_hold_time'] / (1000_000 * max_hold_time_ms),
            'flight_time': X[idx]['flight_time'] / (1000_000*max_flight_time_ms),
            'delta': (X[idx]['delta'] - 1) / 7,
            'local_hour': X[idx]['local_hour'] / 24,
        })
        Ynew.append({
            'src_key': invert_map[Y[idx]['src_key']],
            'dest_key': invert_map[Y[idx]['dest_key']]
        })
    return Xnew, Ynew

In [90]:
X_norm, y_norm = normalize_training_data(X, Y)

In [91]:
X_norm[1000:1005]

[{'src_hold_time': 0.10319375,
  'dest_hold_time': 0.103629375,
  'flight_time': 0.32080916666666665,
  'delta': 0.2857142857142857,
  'local_hour': 0.6666666666666666},
 {'src_hold_time': 0.1045975,
  'dest_hold_time': 0.067177875,
  'flight_time': 0.27646183333333335,
  'delta': 0.2857142857142857,
  'local_hour': 0.6666666666666666},
 {'src_hold_time': 0.104560625,
  'dest_hold_time': 0.210096875,
  'flight_time': 0.5000613333333334,
  'delta': 0.2857142857142857,
  'local_hour': 0.6666666666666666},
 {'src_hold_time': 0.103629375,
  'dest_hold_time': 0.147914,
  'flight_time': 0.4741329166666667,
  'delta': 0.2857142857142857,
  'local_hour': 0.6666666666666666},
 {'src_hold_time': 0.067177875,
  'dest_hold_time': 0.06783075,
  'flight_time': 0.5728454166666667,
  'delta': 0.2857142857142857,
  'local_hour': 0.6666666666666666}]

In [92]:
y_norm[1000:1005]

[{'src_key': 12, 'dest_key': 3},
 {'src_key': 8, 'dest_key': 28},
 {'src_key': 13, 'dest_key': 27},
 {'src_key': 3, 'dest_key': 8},
 {'src_key': 28, 'dest_key': 28}]